In [1]:
from collections import defaultdict
import pandas as pd
from ncaa_pbp import Game
from tqdm import tqdm
from utility import get_Teams

In [23]:
g = Game('3970317',source='file')

In [2]:
team_links = get_Teams(2024)
team_links

{'Kansas City': ('/teams/560547', 'Summit League'),
 'North Florida': ('/teams/560548', 'ASUN'),
 'UIW': ('/teams/560549', 'Southland'),
 'Wofford': ('/teams/560550', 'SoCon'),
 'USC Upstate': ('/teams/560551', 'Big South'),
 'Belmont': ('/teams/560553', 'MVC'),
 'High Point': ('/teams/560554', 'Big South'),
 'A&M-Corpus Christi': ('/teams/560556', 'Southland'),
 'Lipscomb': ('/teams/560559', 'ASUN'),
 'FGCU': ('/teams/560561', 'ASUN'),
 'Utah Valley': ('/teams/560563', 'WAC'),
 'Merrimack': ('/teams/560564', 'NEC'),
 'UC San Diego': ('/teams/560566', 'Big West'),
 'Tarleton St.': ('/teams/560568', 'WAC'),
 'California Baptist': ('/teams/560570', 'WAC'),
 'North Ala.': ('/teams/560572', 'ASUN'),
 'Abilene Christian': ('/teams/560574', 'WAC'),
 'Akron': ('/teams/560576', 'MAC'),
 'Alabama A&M': ('/teams/560578', 'SWAC'),
 'Alabama St.': ('/teams/560580', 'SWAC'),
 'Alabama': ('/teams/560582', 'SEC'),
 'UAB': ('/teams/560584', 'AAC'),
 'UAlbany': ('/teams/560586', 'America East'),
 'Alco

In [3]:
teamNamesMap = {v[0].split('/')[-1]:k for k,v in team_links.items()}

In [4]:
teams_list = list(map(lambda x: x[0].split('/')[-1], team_links.values()))
teams_dict = {team: {'rating':[1, 1], 'poss':[0, 0]} for team in teams_list}


In [5]:
games_df = pd.read_csv('games_2023-24.csv',header=0)
games_df['Date'] = pd.to_datetime(games_df.Date)
games_df = games_df.loc[games_df.duplicated('GameID')]
games_df = games_df.sort_values('Date')
games_df.head()

,Date,Opponent,Result,Attendance,GameID
8037,2023-11-06,"('LIU', '/teams/560846')",W 81-44,"('386', None)",3970317
9295,2023-11-06,"('@ Ball St.', '/teams/560607')",L 64-97,"('4,212', None)",3970228
9233,2023-11-06,"('Delaware St.', '/teams/560713')",W 109-43,"('4,601', None)",3970201
9168,2023-11-06,"('Columbia', '/teams/560692')",W 85-73,"('905', None)",3970204
8970,2023-11-06,"('@ Little Rock', '/teams/560601')",W 52-47,"('2,146', None)",3970250


In [6]:
tol = 0.0001

def adjustRatings(teams: list, points: list, possessions: list):
    poss1 = teams_dict[teams[0]]['poss']
    poss2 = teams_dict[teams[1]]['poss']

    weighted_off1 = teams_dict[teams[0]]['rating'][0] * (poss1[0] / (poss1[0] + possessions[0]))
    weighted_def2 = teams_dict[teams[1]]['rating'][1] * (poss2[1] / (poss2[1] + possessions[0]))

    weighted_def1 = teams_dict[teams[0]]['rating'][1] * (poss1[1] / (poss1[1] + possessions[1]))
    weighted_off2 = teams_dict[teams[1]]['rating'][0] * (poss2[0] / (poss2[0] + possessions[1]))

    new_d = teams_dict[teams[1]]['rating'][1]
    new_o = teams_dict[teams[0]]['rating'][0] # for first iteration

    # if it is a team's first game
    if poss1[0] == 0 or poss2[1] == 0:
        new_o, new_d = ((points[0] / possessions[0]) / new_d) * (possessions[0] / (poss1[0] + possessions[0])) + weighted_off1, \
                        ((points[0] / possessions[0]) / new_o) * (possessions[0] / (poss2[1] + possessions[0])) + weighted_def2
    else:    
        while True:
            old_o = new_o
            new_o = ((points[0] / possessions[0]) / new_d) * (possessions[0] / (poss1[0] + possessions[0])) + weighted_off1
            
            new_d = ((points[0] / possessions[0]) / new_o) * (possessions[0] / (poss2[1] + possessions[0])) + weighted_def2

            if abs(new_o - old_o) < tol:
                break
    teams_dict[teams[0]]['rating'][0] = new_o
    teams_dict[teams[0]]['poss'][0] += possessions[0]
    teams_dict[teams[1]]['rating'][1] = new_d
    teams_dict[teams[1]]['poss'][1] += possessions[0]
    
    new_d = teams_dict[teams[0]]['rating'][1]
    new_o = teams_dict[teams[1]]['rating'][0] # for first iteration
    
    # if it is a team's first game
    if poss2[0] == 0 or poss1[1] == 0:
        new_o, new_d = ((points[1] / possessions[1]) / new_d) * (possessions[1] / (poss2[0] + possessions[1])) + weighted_off2, \
                       ((points[1] / possessions[1]) / new_o) * (possessions[1] / (poss1[1] + possessions[1])) + weighted_def1
    else: 
        while True:
            old_o = new_o
            new_o = ((points[1] / possessions[1]) / new_d) * (possessions[1] / (poss2[0] + possessions[1])) + weighted_off2

            new_d = ((points[1] / possessions[1]) / new_o) * (possessions[1] / (poss1[1] + possessions[1])) + weighted_def1
            
            if abs(new_o - old_o) < tol:
                break
    teams_dict[teams[1]]['rating'][0] = new_o
    teams_dict[teams[1]]['poss'][0] += possessions[1]
    teams_dict[teams[0]]['rating'][1] = new_d
    teams_dict[teams[0]]['poss'][1] += possessions[1]

In [149]:
teams_dict = {'a':{'rating':[1, 1], 'poss':[40, 40]}, 'b':{'rating':[.9, .9], 'poss':[40, 40]}}
adjustRatings(['a','b'],[44,36],[40,40])
teams_dict

{'a': {'rating': [1.070758879569758, 1.0], 'poss': [80, 80]},
 'b': {'rating': [0.9, 0.9636543908195239], 'poss': [80, 80]}}

In [7]:
# Load games 
games = []

for gameid in tqdm(games_df.GameID.tolist()):
    game = Game(gameid,source='file')
    games.append(game)
    teams = game.teams
    points = [game.boxScore[team].PTS.iloc[-1] for team in teams]
    aprox_poss = [game.boxScore[team].FGA.iloc[-1] + game.boxScore[team].TO.iloc[-1]
                  + .475 * game.boxScore[team].FTA.iloc[-1] - game.boxScore[team].ORebs.iloc[-1] for team in game.teams]
    possessions = [(aprox_poss[0] + aprox_poss[1])/2]*2
    adjustRatings(teams, points, possessions)

100%|██████████| 5560/5560 [33:44<00:00,  2.75it/s]  


In [9]:
{teamNamesMap[teamid]: rdict['rating'][0] for teamid, rdict in sorted(teams_dict.items(),key=lambda x: x[1]['rating'][0],reverse=True)}

{'Iowa': np.float64(1.398726446892584),
 'South Carolina': np.float64(1.3418494379687873),
 'UConn': np.float64(1.3324490244003286),
 'Stanford': np.float64(1.3099338379076078),
 'Gonzaga': np.float64(1.3064639766155917),
 'Texas': np.float64(1.2836049329066512),
 'Utah': np.float64(1.2832579433969458),
 'Michigan St.': np.float64(1.2754427414188907),
 'Virginia Tech': np.float64(1.271620727101713),
 'Indiana': np.float64(1.2639030202924624),
 'LSU': np.float64(1.254147427638021),
 'Creighton': np.float64(1.2539670365489086),
 'Ohio St.': np.float64(1.2509311374509733),
 'Southern California': np.float64(1.2419853575225832),
 'Oregon St.': np.float64(1.2355515256690877),
 'UCLA': np.float64(1.2337201001929958),
 'Tennessee': np.float64(1.2276382960590961),
 'Colorado': np.float64(1.2178491015454165),
 'Iowa St.': np.float64(1.2071509030468572),
 'NC State': np.float64(1.2056461060555699),
 'Penn St.': np.float64(1.2030585304325132),
 'Illinois': np.float64(1.1983206536147648),
 'Syracu

In [10]:
{teamNamesMap[teamid]: rdict['rating'][1] for teamid, rdict in sorted(teams_dict.items(),key=lambda x: x[1]['rating'][1],reverse=False)}


{'South Carolina': np.float64(0.6873604862283457),
 'UConn': np.float64(0.7225475722054152),
 'West Virginia': np.float64(0.7392762448374791),
 'Duke': np.float64(0.740038661685604),
 'Kansas St.': np.float64(0.7405028971995495),
 'Notre Dame': np.float64(0.7410583842488228),
 'UCLA': np.float64(0.7456427721230636),
 'NC State': np.float64(0.7460905174142752),
 'Baylor': np.float64(0.7481939824329369),
 'Texas': np.float64(0.7496988591246899),
 'Auburn': np.float64(0.7503674497147448),
 'Stanford': np.float64(0.7627854053073585),
 'Ohio St.': np.float64(0.7772634919476723),
 'Ole Miss': np.float64(0.7803030368009921),
 'Texas A&M': np.float64(0.7806452131130671),
 'Utah': np.float64(0.7840290252012281),
 'Southern California': np.float64(0.7843069959781724),
 'Oregon St.': np.float64(0.7904617966882709),
 'North Carolina': np.float64(0.7922888449207153),
 'Indiana': np.float64(0.7931778005867249),
 'Washington': np.float64(0.7946804871592282),
 'Louisville': np.float64(0.80215991858725

In [11]:
adjEff = pd.DataFrame({team:stats['rating'] for team, stats in teams_dict.items()}).transpose()
adjEff.columns = ['AdjOff','AdjDef']
adjEff['AdjNet'] = adjEff['AdjOff'] - adjEff['AdjDef']
adjEff['Team'] = adjEff.index.map(teamNamesMap)
adjEff = adjEff.sort_values('AdjNet',ascending=False)

adjEff

,AdjOff,AdjDef,AdjNet,Team
561087,1.341849,0.687360,0.654489,South Carolina
560694,1.332449,0.722548,0.609901,UConn
560818,1.398726,0.809241,0.589486,Iowa
561123,1.309934,0.762785,0.547148,Stanford
561155,1.283605,0.749699,0.533906,Texas
...,...,...,...,...
560580,0.692849,1.116875,-0.424026,Alabama St.
561246,0.812609,1.247345,-0.434737,Gardner-Webb
561057,0.741979,1.186382,-0.444403,Saint Francis
560675,0.748430,1.197393,-0.448963,Chicago St.


In [12]:
adjEff.head(20)

,AdjOff,AdjDef,AdjNet,Team
561087,1.341849,0.687360,0.654489,South Carolina
560694,1.332449,0.722548,0.609901,UConn
560818,1.398726,0.809241,0.589486,Iowa
561123,1.309934,0.762785,0.547148,Stanford
561155,1.283605,0.749699,0.533906,Texas
561183,1.283258,0.784029,0.499229,Utah
560661,1.233720,0.745643,0.488077,UCLA
560993,1.250931,0.777263,0.473668,Ohio St.
560811,1.263903,0.793178,0.470725,Indiana
560962,1.205646,0.746091,0.459556,NC State


In [13]:
netRankings = pd.read_html("https://stats.ncaa.org/selection_rankings/nitty_gritties/37268")[0]
netRankings.sort_values('NET Rank', inplace=True)
net_map = dict(zip(netRankings.Team,netRankings['NET Rank']))
conf_map = dict(zip(netRankings.Team,netRankings['Conference']))
net_map

{'South Carolina': 1,
 'UConn': 2,
 'Texas': 3,
 'Iowa': 4,
 'Stanford': 5,
 'UCLA': 6,
 'LSU': 7,
 'Notre Dame': 8,
 'Ohio St.': 9,
 'Southern California': 10,
 'NC State': 11,
 'Utah': 12,
 'Indiana': 13,
 'Gonzaga': 14,
 'Colorado': 15,
 'Oregon St.': 16,
 'Baylor': 17,
 'Kansas St.': 18,
 'Virginia Tech': 19,
 'Duke': 20,
 'West Virginia': 21,
 'Michigan St.': 22,
 'Creighton': 23,
 'Louisville': 24,
 'Nebraska': 25,
 'Tennessee': 26,
 'Oklahoma': 27,
 'UNLV': 28,
 'Penn St.': 29,
 'Alabama': 30,
 'Iowa St.': 31,
 'Maryland': 32,
 'Washington St.': 33,
 'Ole Miss': 34,
 'Princeton': 35,
 'North Carolina': 36,
 'Arizona': 37,
 'Kansas': 38,
 'Texas A&M': 39,
 'Middle Tenn.': 40,
 'Illinois': 41,
 'Marquette': 42,
 'Mississippi St.': 43,
 'Syracuse': 44,
 'Florida St.': 45,
 'Villanova': 46,
 'Auburn': 47,
 'TCU': 48,
 'Michigan': 49,
 'Washington': 50,
 'Miami (FL)': 51,
 'Richmond': 52,
 'Green Bay': 53,
 'FGCU': 54,
 'Vanderbilt': 55,
 'Columbia': 56,
 'South Dakota St.': 57,
 'Ok

In [14]:
adjEff['NET'] = adjEff['Team'].map(net_map)
adjEff['Rank'] = range(1,len(adjEff.index)+1)
adjEff['Diff'] = adjEff['NET'] - adjEff['Rank']
adjEff['Conference'] = adjEff['Team'].map(conf_map)
adjEff.to_csv('adjusted_efficiency_2023_24.csv')
adjEff.tail(20)

,AdjOff,AdjDef,AdjNet,Team,NET,Rank,Diff,Conference
561105,0.878161,1.181161,-0.303000,SIUE,327,341,-14,OVC
560793,0.735261,1.038489,-0.303229,Houston Christian,348,342,6,Southland
560848,0.782515,1.096439,-0.313924,Longwood,341,343,-2,Big South
561149,0.763265,1.095540,-0.332275,Texas Southern,340,344,-4,SWAC
561261,0.843634,1.178140,-0.334506,Lindenwood,325,345,-20,OVC
560925,0.768657,1.118370,-0.349712,UNC Asheville,335,346,-11,Big South
561211,0.784113,1.136536,-0.352423,Western Caro.,350,347,3,SoCon
561266,0.740670,1.105657,-0.364987,Stonehill,358,348,10,NEC
560882,0.817754,1.184964,-0.367210,McNeese,353,349,4,Southland
560932,0.746940,1.124481,-0.377541,UNCW,352,350,2,CAA


In [15]:
adjEff.sort_values(['Conference','Rank'])

,AdjOff,AdjDef,AdjNet,Team,NET,Rank,Diff,Conference
561135,1.053581,0.863128,0.190453,Temple,92,75,17,AAC
560969,1.046957,0.894764,0.152193,North Texas,85,87,-2,AAC
561041,1.021005,0.870743,0.150262,Rice,95,88,7,AAC
561173,1.047000,0.910834,0.136166,Tulsa,106,95,11,AAC
560728,0.971930,0.863700,0.108230,East Carolina,116,111,5,AAC
...,...,...,...,...,...,...,...,...
561073,0.966655,0.941885,0.024770,San Francisco,159,153,6,WCC
561064,0.979628,0.993889,-0.014261,Saint Mary's (CA),208,169,39,WCC
561071,0.958814,0.978966,-0.020151,San Diego,210,171,39,WCC
560860,0.955297,1.012927,-0.057630,LMU (CA),219,197,22,WCC


**Rebounding**

In [16]:
reb_dict = {team: {'rating':[1, 1], 'poss':[0, 0]} for team in teams_list}

In [394]:
# Adjust rebound
tol = 0.0001
nationalAverage = .3

def adjustReb(teams: list, rebounds: list, possessions: list):
    poss1 = reb_dict[teams[0]]['poss']
    poss2 = reb_dict[teams[1]]['poss']

    weighted_off1 = reb_dict[teams[0]]['rating'][0] * (poss1[0] / (poss1[0] + possessions[0]))
    weighted_def2 = reb_dict[teams[1]]['rating'][1] * (poss2[1] / (poss2[1] + possessions[0]))

    weighted_def1 = reb_dict[teams[0]]['rating'][1] * (poss1[1] / (poss1[1] + possessions[1]))
    weighted_off2 = reb_dict[teams[1]]['rating'][0] * (poss2[0] / (poss2[0] + possessions[1]))

    new_d = reb_dict[teams[1]]['rating'][1]
    new_o = reb_dict[teams[0]]['rating'][0] # for first iteration

    # if it is a team's first game
    if poss1[0] == 0 or poss2[1] == 0:
        new_o, new_d = ((rebounds[0] / possessions[0]) / new_d)*nationalAverage * (possessions[0] / (poss1[0] + possessions[0])) + weighted_off1, \
                        ((rebounds[0] / possessions[0]) / new_o)*nationalAverage * (possessions[0] / (poss2[1] + possessions[0])) + weighted_def2
    else:    
        while True:
            old_o = new_o
            new_o = ((rebounds[0] / possessions[0]) / new_d)*nationalAverage * (possessions[0] / (poss1[0] + possessions[0])) + weighted_off1
            
            new_d = ((rebounds[0] / possessions[0]) / new_o)*nationalAverage * (possessions[0] / (poss2[1] + possessions[0])) + weighted_def2

            if abs(new_o - old_o) < tol:
                break
    reb_dict[teams[0]]['rating'][0] = new_o
    reb_dict[teams[0]]['poss'][0] += possessions[0]
    reb_dict[teams[1]]['rating'][1] = new_d
    reb_dict[teams[1]]['poss'][1] += possessions[0]
    
    new_d = reb_dict[teams[0]]['rating'][1]
    new_o = reb_dict[teams[1]]['rating'][0] # for first iteration
    
    # if it is a team's first game
    if poss2[0] == 0 or poss1[1] == 0:
        new_o, new_d = ((rebounds[1] / possessions[1]) / new_d)*nationalAverage * (possessions[1] / (poss2[0] + possessions[1])) + weighted_off2, \
                       ((rebounds[1] / possessions[1]) / new_o)*nationalAverage * (possessions[1] / (poss1[1] + possessions[1])) + weighted_def1
    else: 
        while True:
            old_o = new_o
            new_o = ((rebounds[1] / possessions[1]) / new_d)*nationalAverage * (possessions[1] / (poss2[0] + possessions[1])) + weighted_off2

            new_d = ((rebounds[1] / possessions[1]) / new_o)*nationalAverage * (possessions[1] / (poss1[1] + possessions[1])) + weighted_def1
            
            if abs(new_o - old_o) < tol:
                break
    reb_dict[teams[1]]['rating'][0] = new_o
    reb_dict[teams[1]]['poss'][0] += possessions[1]
    reb_dict[teams[0]]['rating'][1] = new_d
    reb_dict[teams[0]]['poss'][1] += possessions[1]

In [395]:

for game in tqdm(games):

    teams = game.teams
    rebs = [game.boxScore[team].ORebs.iloc[-1] for team in teams]
    chances = [game.boxScore[team].FGA.iloc[-1] - game.boxScore[team].FGM.iloc[-1] for team in game.teams]
    adjustReb(teams, rebs, chances)

  0%|          | 0/5560 [00:00<?, ?it/s]

100%|██████████| 5560/5560 [00:02<00:00, 2373.81it/s]


In [396]:
{teamNamesMap[teamid]: rdict['rating'][0] for teamid, rdict in sorted(reb_dict.items(),key=lambda x: x[1]['rating'][0],reverse=True)}

{'Texas': np.float64(0.628238118046574),
 'LSU': np.float64(0.486176880325215),
 'Oklahoma': np.float64(0.4801217030405596),
 'South Carolina': np.float64(0.4792845051747429),
 'UCLA': np.float64(0.47363325852621563),
 'Ole Miss': np.float64(0.47183366260700765),
 'Fordham': np.float64(0.46902398746449225),
 'Duke': np.float64(0.46649794375408454),
 'Colorado': np.float64(0.4619266666396716),
 'San Diego': np.float64(0.4618785547568011),
 'Syracuse': np.float64(0.4607488273407671),
 'Gonzaga': np.float64(0.4578285517676365),
 'Stanford': np.float64(0.4418071742451129),
 'Rhode Island': np.float64(0.42696785588089536),
 'Saint Louis': np.float64(0.4264327994589993),
 'Miami (FL)': np.float64(0.42513436674532845),
 'Villanova': np.float64(0.4238819251579062),
 'Louisville': np.float64(0.42379043372412556),
 'UTSA': np.float64(0.4236231208828123),
 'Virginia': np.float64(0.4229376877813068),
 'Seattle U': np.float64(0.42021113946624855),
 'Providence': np.float64(0.41985881439129963),
 'C

In [399]:
adjReb = pd.DataFrame({team:stats['rating'] for team, stats in reb_dict.items()}).transpose()
adjReb.columns = ['AdjOffReb','AdjDefReb']
adjReb['AdjDefReb'] = 1 - adjReb['AdjDefReb']
adjReb['Team'] = adjReb.index.map(teamNamesMap)
adjReb = adjReb.sort_values('AdjDefReb',ascending=False)
adjReb.to_csv('adjusted_rebounding_2023_24.csv')
adjReb.head(20)

,AdjOffReb,AdjDefReb,Team
561183,0.356022,0.817434,Utah
560938,0.281680,0.814855,UNLV
560962,0.337779,0.813813,NC State
560816,0.367380,0.806594,Iowa St.
561005,0.312006,0.805383,Oregon St.
560661,0.473633,0.801813,UCLA
560627,0.383757,0.794305,BYU
560611,0.411391,0.791605,Baylor
560934,0.403859,0.791312,Nebraska
560701,0.285383,0.788702,Creighton


In [21]:
teams = game.teams
rebs = [game.boxScore[team].ORebs.iloc[-1] for team in teams]
rebs

[np.int64(7), np.int64(18)]

In [23]:
game.boxScore['560818']

,#,Name,P,MP,GS,FGM,FGA,3FG,3FGA,FT,...,ORebs,DRebs,Tot Reb,AST,TO,STL,BLK,PF,DQ,Tech Fouls
0,3.0,Sydney Affolter,G,34:51,1,4,8,1,3,3,...,0,3,3,3,2,3,0,3,0,0
1,22.0,Caitlin Clark,G,39:40,1,10,28,5,13,5,...,2,6,8,5,4,0,0,3,0,0
2,1.0,Molly Davis,G,0:20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,34.0,AJ Ediger,F,0:20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.0,Kylie Feuerbach,G,5:09,0,0,1,0,0,0,...,0,1,1,0,0,0,0,0,0,0
5,40.0,Sharon Goodman,C,0:20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,24.0,Gabbie Marshall,G,39:40,1,2,3,2,3,0,...,0,3,3,1,0,3,0,0,0,0
7,20.0,Kate Martin,G,39:40,1,5,13,1,4,5,...,1,4,5,0,3,0,0,4,0,0
8,2.0,Taylor McCabe,G,0:20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,44.0,Addison O'Grady,C,12:55,0,0,2,0,0,0,...,1,1,2,2,0,0,2,2,0,0


In [20]:
reb = pd.read_csv('adjusted_rebounding_2023_24.csv',index_col=0)
reb = reb.sort_values('AdjDefReb',ascending=False)
reb.reset_index(inplace=True)

reb[reb.Team.str.contains('FGCU|Richmond|Saint Louis|Canyon|Fairfield|Marshall')]

,index,AdjOffReb,AdjDefReb,Team
41,560561,0.311668,0.755701,FGCU
116,561043,0.277714,0.710028,Richmond
261,561248,0.335268,0.652717,Grand Canyon
271,560745,0.350154,0.648168,Fairfield
321,560872,0.375851,0.616452,Marshall
323,561062,0.426433,0.610726,Saint Louis


In [342]:
reb.head(20)

,index,AdjOffReb,AdjDefReb,Team
0,561183,0.356022,0.817434,Utah
1,560938,0.281680,0.814855,UNLV
2,560962,0.337779,0.813813,NC State
3,560816,0.367380,0.806594,Iowa St.
4,561005,0.312006,0.805383,Oregon St.
5,560661,0.473633,0.801813,UCLA
6,560627,0.383757,0.794305,BYU
7,560611,0.411391,0.791605,Baylor
8,560934,0.403859,0.791312,Nebraska
9,560701,0.285383,0.788702,Creighton


In [19]:
games[0].pbp.items[0].currentLineups

{'560846': ['Ashley Austin',
  'Dia Denes',
  'Mariah Elohim',
  'Nigeria Harkless',
  "Emaia O'Brien"],
 '561058': ['Jillian Archer',
  'Amber Brown',
  'Jailah Donald',
  'Unique Drake',
  "Ber'Nyah Mayo"]}